# Prepare and merge previously classified files

In [1]:
import os
import sys
import numpy as np
import pandas as pd

# sys.path.append(os.path.abspath("/Users/urimarom/PycharmProjects/fossil_classification"))
# from fossil_classification import id_clean_col

In [2]:
prev_class_path = "/Users/urimarom/Downloads/חשיפה לפוסיליים - סיווגי רבעונים קודמים - up to 2020Q4.csv"
prev_fossil_classification = pd.read_csv(prev_class_path)
prev_fossil_classification.head()

,Name,security_num,issuer_or_corp_num,is_fossil
0,מץד'טפ_ למט_,1123355,513901371,0.0
1,. אנלייט אנרגיה מתחדשת בעמ,720011,520041146,0.0
2,. אנרג'יקס-אנרגיות מתחדשות,1123355,513901371,0.0
3,. ויקטורי רשת סופרמרקטים בעמ,1123777,514068980,0.0
4,. פטרוכימים אגח ב,7560048,520029315,1.0


In [3]:
pd.set_option('display.max_rows', None)

In [4]:
def id_col_clean(col):
    new_col = pd.Series(col.astype(str).str.strip().str.upper())
# remove '0' and other non valid values
    new_col = new_col.apply(
        lambda x: None if (x == '0') | (x == 'NAN') | (x =='NONE') | (x == '') else x
    )
    return new_col

def fetch_latest_prev_classified(prev_class_path):
    prev_fossil_classification = pd.read_csv(prev_class_path)
    return prev_fossil_classification

def prepare_prev_class(prev_class):
    prev_class["security_num"] = id_col_clean(prev_class["security_num"])
    prev_class["issuer_or_corp_num"] = id_col_clean(prev_class["issuer_or_corp_num"])
    print("\n** Previously classified ISINs and corps **")
    print("is_fossil in previously classified")
    print(prev_class["is_fossil"].value_counts(dropna=False))
    prev_grouped_by_sec_num = prev_class.groupby("security_num")
    sec_num_with_diff_class = prev_grouped_by_sec_num.filter(lambda x:
                                                             (0 < x["is_fossil"].mean() < 1)
                                                             )
# TODO: raise warning for ambiguously classified
    if len(sec_num_with_diff_class) > 0:
        print("\n*** Securities with both is_fossil=1 and is_fossil=0 ***")
        print(sec_num_with_diff_class)

    prev_grouped_by_issuer = prev_class[prev_class["issuer_or_corp_num"] != 'NAN'].groupby("issuer_or_corp_num")
    issuer_with_diff_class = prev_grouped_by_issuer.filter(lambda x:
                                                           (0 < x["is_fossil"].mean() < 1)
                                                           )
    if len(issuer_with_diff_class) > 0:
        print("\n*** Issuers with both is_fossil=1 and is_fossil=0 ***")
        print(issuer_with_diff_class)
    return prev_class

prev_class = prepare_prev_class(fetch_latest_prev_classified(prev_class_path))


** Previously classified ISINs and corps **
is_fossil in previously classified
0.0    16413
1.0     1665
Name: is_fossil, dtype: int64
